In [33]:
%reload_ext autoreload
%autoreload 2
from pywinusb import hid
from lib.DMDpycrafter import DMDhid
import PIL.Image
import matplotlib.pyplot as plt
import numpy as np


In [34]:
images=[]
img = np.asarray(PIL.Image.open("testimage.tif"))//129

h = (1600-1080)/2
w = (2560-1920)/2
img = np.pad(img, ((int(np.floor(h)), int(np.ceil(h))), (int(np.floor(w)), int(np.ceil(w)))), 'constant', constant_values=(0.5, 0.5))

images.append(img)

In [35]:
dmd = DMDhid()
dmd.stopsequence()
dmd.changemode(3)
exposure=[1000000]*30
dark_time=[0]*30
trigger_in=[False]*30
trigger_out=[1]*30
dmd.defsequence(images,exposure,trigger_in,dark_time,trigger_out,0)


merging...
encoding...
uploading...
0 272
100 272
200 272


In [30]:
dmd.startsequence()

In [48]:
img

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [50]:

# image content
for i in range(img.shape[0]):
    # bool array indicating if same as previous row, shape = (1920, )
    same_prev = np.zeros(img.shape[1], dtype=bool) if i == 0 else img[i] == img[i-1]
    # encoded += encode_row(image[i], same_prev)

In [64]:
rr = np.zeros(img.shape[1], dtype=bool) if i == 1 else img[800] == img[799]

In [70]:
prev = img[799]

In [71]:
np.logical_not(np.diff(rr))

array([ True,  True,  True, ...,  True,  True,  True])

In [74]:
sum(rr)

2539

In [78]:
sum(np.logical_not(np.diff(rr)))

2533

In [76]:
np.diff(rr)

array([False, False, False, ..., False, False, False])

In [15]:
from utils import CustomLogger

logger = CustomLogger().info_logger

In [59]:
import struct
pack32be = struct.Struct('>I').pack

In [24]:
def defsequence(self,images,exp,ti,dt,to,rep):

    self.stopsequence()

    arr=[]

    for i in images:
        arr.append(i)

##        arr.append(numpy.ones((1080,1920),dtype='uint8'))

    num=len(arr)

    encodedimages=[]
    sizes=[]

    for i in range((num-1)//24+1):
        print ('merging...')
        if i<((num-1)//24):
            imagedata=arr[i*24:(i+1)*24]
        else:
            imagedata=arr[i*24:]
        print ('encoding...')
        imagedata,size=encode(imagedata)
        

        encodedimages.append(imagedata)
        sizes.append(size)

        if i<((num-1)//24):
            for j in range(i*24,(i+1)*24):
                self.definepattern(j,exp[j],1,'111',ti[j],dt[j],to[j],i,j-i*24)
        else:
            for j in range(i*24,num):
                self.definepattern(j,exp[j],1,'111',ti[j],dt[j],to[j],i,j-i*24)

    self.configurelut(num,rep)

    for i in range((num-1)//24+1):
    
        self.setbmp((num-1)//24-i,sizes[(num-1)//24-i])

        print ('uploading...')
        self.bmpload(encodedimages[(num-1)//24-i],sizes[(num-1)//24-i])

In [25]:
dev = devices[0]

In [26]:
dev.open()

In [27]:
dev.find_output_reports()

[HID report object (Output report, id=0x00), 1 items included]

In [14]:
def bitstobytes(a):
    bytelist=[]
    if len(a)%8!=0:
        padding=8-len(a)%8
        a='0'*padding+a
    for i in range(len(a)//8):
        bytelist.append(int(a[8*i:8*(i+1)],2))

    bytelist.reverse()

    return bytelist

In [4]:
def bitstobytes(a):
    bytelist=[]
    if len(a)%8!=0:
        padding=8-len(a)%8
        a='0'*padding+a
    for i in range(len(a)//8):
        bytelist.append(int(a[8*i:8*(i+1)],2))

    bytelist.reverse()

    return bytelist

def convlen(a,l):
    b=bin(a)[2:]
    padding=l-len(b)
    b='0'*padding+b

    return b

bitstobytes(convlen(len([0xff,0x01,0xff,0x01])+2,16))

[6, 0]

In [ ]:
def command(self,mode,sequencebyte,com1,com2,data=None):
    buffer = []

    flagstring=''
    if mode=='r':
        flagstring+='1'
    else:
        flagstring+='0'        
    flagstring+='1000000'
    buffer.append(bitstobytes(flagstring)[0])
    buffer.append(sequencebyte)
    temp=bitstobytes(convlen(len(data)+2,16))
    buffer.append(temp[0])
    buffer.append(temp[1])
    buffer.append(com2)
    buffer.append(com1)

    if len(buffer)+len(data)<65:
    
        for i in range(len(data)):
            buffer.append(data[i])

        for i in range(64-len(buffer)):
            buffer.append(0x00)


        self.dev.write(1, buffer)

    else:
        for i in range(64-len(buffer)):
            buffer.append(data[i])

        self.dev.write(1, buffer)

        buffer = []

        j=0
        while j<len(data)-58:
            buffer.append(data[j+58])
            j=j+1
            if j%64==0:
                self.dev.write(1, buffer)

                buffer = []

        if j%64!=0:

            while j%64!=0:
                buffer.append(0x00)
                j=j+1


            self.dev.write(1, buffer)                
            





    self.ans=self.dev.read(0x81,64)

In [17]:
bitstobytes(convlen(len([int('00000001',2)])+2,16))

[3, 0]

In [ ]:
def convlen(a,l):
    b=bin(a)[2:]
    padding=l-len(b)
    b='0'*padding+b

    return b

def bitstobytes(a):
    bytelist=[]
    if len(a)%8!=0:
        padding=8-len(a)%8
        a='0'*padding+a
    for i in range(len(a)//8):
        bytelist.append(int(a[8*i:8*(i+1)],2))

    bytelist.reverse() 


In [45]:
for i in range(-10):
    print(i)

In [23]:
len([0xff,0x01,0xff,0x01,0xff,0x01])

6

In [29]:
convlen(3, 16)

'0000000000000011'

In [33]:
bitstobytes(convlen(3, 16))

[3, 0]

In [34]:
for i in range(0):
    print(i)

In [1]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

from lib.utils import CustomLogger
import logging

In [2]:
logger = CustomLogger().info_logger

In [3]:
def asdf():
    for i in range(1):
        logger.info('asdf')
asdf()

[INFO | 2022-02-19 20:35:54,826 | 1375158448.py | asdf] (line 3) asdf :: info logger
